# Zbiór Nauka
## inf132326, inf132328
Given is a set of 18 countries for which we need to evaluate their scientific wealth. They are evaluated using 2 inputs and 2 outputs descibled below.

Inputs:
- i1: GDP (in 109 dollars);
- i2: active population (in millions);
- i3: R&D expenditure (in 106 dollars); 

Outputs:
- o1: number of publications in the SCI (1993);
- o2: number of patents granted by the European Patent OfficeS(1993);

### Ładowanie biblioteki:
Wczytujemy bibliotekę lpSolveAPI

In [1]:
library(lpSolveAPI)

### Tabela wejść:
Pobieramy dane wejściowe i wpisujemy do tabeli *inputs*:

In [2]:
inputs <- read.csv("inputs.csv", header = TRUE, sep=";", row.names = 1)
inputs

,i1,i2,i3
,<int>,<int>,<int>
Austria,184,3880,1975
Belgium,219,4237,3189
Denmark,142,2903,2206
Finland,109,2502,2271
France,1324,25385,28907
Germany,1939,39628,38609
Ireland,50,1375,299
Italy,1224,23210,14190
Netherlands,320,7011,5581


### Tabela wyjść
Pobieramy dane wyjściowe i wpisujemy do tabeli *outputs*:

In [3]:
outputs <- read.csv("outputs.csv", header = TRUE, sep=";", row.names = 1)
outputs

,o1,o2
,<int>,<int>
Austria,4322,438
Belgium,6681,263
Denmark,5892,231
Finland,4685,170
France,35612,3355
Germany,44173,8736
Ireland,1512,31
Italy,21505,1269
Netherlands,15110,1154


### Wyznaczenie wymiarów modelu:
Wyznaczamy wymiary naszego modelu i zapisujemy do zmiennej *science_dim*.

W naszym przypadku jej wartość to **liczba wierszy z tabeli wejść plus theta** *(18 + 1 = 19)*.

In [4]:
science_dim <- nrow(inputs) + 1

### Określenie wymiarów modelu:
Funkcja, która utworzy model dla podanego wymiaru. 

In [5]:
createModel = function()
{
    return(make.lp(ncol = science_dim))
}

### Określenie kierunku optymalizacji:
Funkcja, która określi kierunek optymalizacji naszego modelu na **minimalizację**.

In [6]:
setDirection = function(model) 
{
    lp.control(model, sense = "min")
}

### Określenie funkcji celu:
Funkcja, która określi co chcemy minimalizować. 
W naszym przypadku jest to tylko **theta**, czyli **pierwsza zmienna**.

In [7]:
setObjective = function(model, inputs, outputs, country)
{
    objective_coeffs = 1:1
    set.objfn(model, objective_coeffs, indices = 1:1) 
}

### Dodanie pierwszych ograniczeń
Funkcja dodająca pierwsze ograniczenia. Opis z karty ćwiczeń przerobionej na laboratoriach: 
> **combination of inputs not greater than analyzed DMU's multiplied by $\theta$**.

In [8]:
addThetaConstraints = function(model, inputs, outputs, country)
{
    for (i in 2:ncol(inputs))
    {
        constraint_coeffs = array(0, dim = science_dim)
        constraint_coeffs[1] = -1 * inputs[country, i]
        constraint_coeffs[2:science_dim] = unlist(inputs[, i])
        add.constraint(model, constraint_coeffs, "<=", 0)
    } 
}

### Dodanie drugich ograniczeń:
Funkcja dodające drugie ograniczenia. Opis z karty przerobionej na laboratoriach:
> **combination of output not less than analyzed DMU's**.

In [9]:
addOutputConstraints = function(model, inputs, outputs, country)
{
    for(i in 2:ncol(outputs))
    {
        constraint_coeffs = array(0, dim = science_dim)
        constraint_coeffs[2:science_dim] = unlist(outputs[, i])
        add.constraint(model, constraint_coeffs, ">=", outputs[country, i])
    }
}

### Tworzenie całego modelu
Funkcja, która stworzy model o odpowiednich wymiarach, nada kierunek optymalizacji i funkcję celu oraz doda ograniczenia.

In [10]:
createScienceCCRModel = function(inputs, outputs, country)
{
    model = createModel()
    setDirection(model)
    setObjective(model, inputs, outputs, country)
    addThetaConstraints(model, inputs, outputs, country)
    addOutputConstraints(model, inputs, outputs, country)
    return(model)
}

### Wyniki
Iterujemy po wszystkich krajach i obliczami ich efektywność. Dla jednostek nieefektywnych obliczami o ile muszą się poprawić - zmniejszyć input - zgdonie ze wzorem:
> $original\_input \cdot (1 - \theta)$

In [11]:
for (country in 1:18)
{
    model <- createScienceCCRModel(inputs, outputs, country)
    solve(model)
    
    efficiency <- get.objective(model)
    lambdas <-  get.variables(model)
    
    cat(rownames(inputs)[country], "\n  efektywność =", efficiency, "\n")
    for (lamb in 2:length(lambdas))
    {
        if (lambdas[lamb] > 0)
        {
            cat("  lambda", rownames(inputs)[lamb - 1], lambdas[lamb])
        }
    }
    cat("\n")
    
    if (efficiency < 0.9999999)
    {
        for(in_var in 1:3)
        {
            cat(" ", inputs[country, in_var] * (1 - efficiency))
        }
    }
    cat("\n\n")
}

Austria 
  efektywność = 0.9801283 
  lambda Germany 0.05013736
  3.656389  77.10211  39.24657

Belgium 
  efektywność = 0.3644829 
  lambda Germany 0.03010531
  139.1782  2692.686  2026.664

Denmark 
  efektywność = 0.4627883 
  lambda Germany 0.02644231
  76.28406  1559.525  1185.089

Finland 
  efektywność = 0.3308322 
  lambda Germany 0.01945971
  72.9393  1674.258  1519.68

France 
  efektywność = 0.5301964 
  lambda Germany 0.2713514  lambda Switzerland 0.6913443
  622.02  11925.97  13580.61

Germany 
  efektywność = 1 
  lambda Germany 1


Ireland 
  efektywność = 0.458212 
  lambda Germany 0.003548535
  27.0894  744.9585  161.9946

Italy 
  efektywność = 0.3952348 
  lambda Germany 0.145261
  740.2326  14036.6  8581.618

Netherlands 
  efektywność = 0.9138391 
  lambda Germany 0.1320971
  27.5715  604.0744  480.8642

Norway 
  efektywność = 0.2287916 
  lambda Germany 0.01156136
  87.14655  1658.869  1504.628

Spain 
  efektywność = 0.08821843 
  lambda Germany 0.009386447
  43